In [1]:
import numpy as np
import _pickle as cp
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.core import Activation
from keras.layers import concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import Conv3D
from keras.layers.normalization import BatchNormalization
from keras.layers import Input
from keras import optimizers
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Reshape
import keras.backend as K
from keras.layers.recurrent import GRU,LSTM
from keras.layers import GRUCell,Lambda
from keras.layers import RNN
from keras import regularizers

from sliding_window2 import sliding_window

Using TensorFlow backend.


In [41]:
import pandas as pd

NB_SENSOR_CHANNELS =  6 # originla 6
NUM_CLASSES = 6
SLIDING_WINDOW_LENGTH = 24 #original 24
FINAL_SEQUENCE_LENGTH = 8
SLIDING_WINDOW_STEP = 12
BATCH_SIZE = 100
NUM_FILTERS = 64
FILTER_SIZE = 5
NUM_UNITS_LSTM = 128

df = pd.read_csv('all_raw_data.csv')
print(df.head())
df.drop('Unnamed: 0',axis='columns',inplace=True)
# df.rename(columns={ 'x_g.1':'z_g'}, inplace = True)
print(df.shape)

# df.dropna(subset=['Index','gt'], inplace=True)
print(df.isnull().sum(axis=0))

x_train = df
y_train = pd.read_csv('all_label.csv')
y_train.drop('Unnamed: 0',axis='columns',inplace=True)
print(y_train.head())
x_train = x_train.iloc[:2400000].values
print(x_train.shape)
y_train = y_train.iloc[:2400000].values
x_train = x_train.reshape((-1,6*24))
y_train = y_train.reshape((-1,6*24))
print(x_train.shape,y_train.shape)

   Unnamed: 0     acc_x     acc_y     acc_z    gyro_x    gyro_y    gyro_z
0           0 -6.090849  0.565032  7.623138 -0.006109  0.010690 -0.006720
1           1 -6.090849  0.593762  7.603985  0.001222  0.011301 -0.006414
2           2 -6.100426  0.612916  7.651869  0.000305  0.015577 -0.008247
3           3 -6.033388  0.603339  7.651869  0.004581  0.007941 -0.000611
4           4 -6.081272  0.641646  7.671022  0.013134  0.015577 -0.001527
(2548244, 6)
acc_x     0
acc_y     0
acc_z     0
gyro_x    0
gyro_y    0
gyro_z    0
dtype: int64
   bike  sit  stairsdown  stairsup  stand  walk
0     0    0           0         0      1     0
1     0    0           0         0      1     0
2     0    0           0         0      1     0
3     0    0           0         0      1     0
4     0    0           0         0      1     0
(2400000, 6)
(100000, 144) (100000, 144)


In [42]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [43]:

assert NB_SENSOR_CHANNELS == x_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x, (ws, data_x.shape[1]), (ss, 1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y, ws, ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
x_train, y_train = opp_sliding_window(x_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
x_test, y_test = opp_sliding_window(x_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)

# Data is reshaped
x_train = x_train.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)) # for input to Conv1D
x_test = x_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS)) # for input to Conv1D
y_train = np_utils.to_categorical(y_train) # one-hot encoding
y_test = np_utils.to_categorical(y_test) # one-hot encoding

print(" ..after sliding and reshaping, train data: inputs {0}, targets {1}".format(x_train.shape, y_train.shape))
print(" ..after sliding and reshaping, test data : inputs {0}, targets {1}".format(x_test.shape, y_test.shape))

ValueError: a.shape, ws and ss must all have the same length. They were [2, 1, 1]

In [5]:
n_hidden = 64 # Hidden layer num of features
n_classes = 6 # Total classes (should go up, or should go down)


# Training 

learning_rate = 0.015
lambda_loss_amount = 0.015
batch_size = 1500
display_iter = 30000  # To show test set accuracy during training


sensor_input = Input(shape=(24,6), name='input')
outputs = LSTM(n_hidden,unit_forget_bias=True,return_sequences=True)(sensor_input)
outputs = LSTM(n_hidden,unit_forget_bias=True,return_sequences=True)(outputs)

# def reduce_dim(x):
#     return x[:,-1:,:]

# outputs = Lambda(reduce_dim)(outputs)
outputs = Activation('linear')(outputs)
outputs = Flatten()(outputs)

outputs = Dense(6, activation='softmax',kernel_regularizer=regularizers.l2(lambda_loss_amount))(outputs)
# keras.regularizers.l2(lambda_loss_amount)
adam = keras.optimizers.Adam(lr=learning_rate,epsilon=None, decay=0.0, amsgrad=False)

model = Model(inputs=sensor_input, outputs=outputs)
model.compile(loss='categorical_crossentropy',
      optimizer=adam,
      metrics=['accuracy'])

#model.fit([acc_inputs,gyro_inputs], train_y, batch_size=64, epochs=10)
model.summary()


W0906 15:00:23.258462  2908 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0906 15:00:23.275463  2908 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0906 15:00:23.277464  2908 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0906 15:00:23.865497  2908 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimize

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 24, 6)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 24, 64)            18176     
_________________________________________________________________
lstm_2 (LSTM)                (None, 24, 64)            33024     
_________________________________________________________________
activation_1 (Activation)    (None, 24, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1536)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 9222      
Total params: 60,422
Trainable params: 60,422
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(x_train, y_train, batch_size=128,validation_split=0.2, epochs=10)

W0906 15:00:24.088510  2908 deprecation.py:323] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0906 15:00:25.635599  2908 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 114770 samples, validate on 28693 samples
Epoch 1/10
114770/114770 [==============================] - 159s 1ms/step - loss: 1.4858 - acc: 0.3659 - val_loss: 1.2865 - val_acc: 0.4818
Epoch 2/10
114770/114770 [==============================] - 156s 1ms/step - loss: 1.2623 - acc: 0.4886 - val_loss: 1.2200 - val_acc: 0.5116
Epoch 3/10
114770/114770 [==============================] - 157s 1ms/step - loss: 1.2212 - acc: 0.5134 - val_loss: 1.2228 - val_acc: 0.5193
Epoch 4/10
114770/114770 [==============================] - 157s 1ms/step - loss: 1.1934 - acc: 0.5291 - val_loss: 1.1806 - val_acc: 0.5335
Epoch 5/10
114770/114770 [==============================] - 158s 1ms/step - loss: 1.1746 - acc: 0.5394 - val_loss: 1.1612 - val_acc: 0.5447
Epoch 6/10
114770/114770 [==============================] - 158s 1ms/step - loss: 1.1537 - acc: 0.5503 - val_loss: 1.1558 - val_acc: 0.5531
Epoch 7/10
114770/114770 [==============================] - 157s 1ms/step - loss: 1.1553 - acc: 0.5539 - val_

In [8]:
from keras.models import load_model
 
model.save('LSTM_keras_raw.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model
 
# returns a compiled model
# identical to the previous one
# model = load_model('LSTM_keras_raw.h5')

In [17]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn import metrics
test_x = x_test
test_y = y_test

# print ( "Loss = " + str( preds[0] ) )
# print ( "Test Accuracy = " + str( preds[1] ) )
print(model.metrics_names)
loss, acc = model.evaluate(test_x, test_y, batch_size=128)
print('loss, acc ',loss,acc)


y_pred = model.predict(test_x, batch_size=128, verbose=1)
# y_pred_bool = np.argmax(preds, axis=1)

for i in range(len(y_pred)):
        max_value=max(y_pred[i])
        for j in range(len(y_pred[i])):
            if max_value==y_pred[i][j]:
                y_pred[i][j]=1
            else:
                y_pred[i][j]=0

print(classification_report(test_y, y_pred))
print(metrics.f1_score(test_y, y_pred, average="weighted"))

['loss', 'acc']
70660/70660 [==============================] - 29s 414us/step
loss, acc  1.1338554639368195 0.5594820266062837
70660/70660 [==============================] - 29s 405us/step
              precision    recall  f1-score   support

           0       0.55      0.64      0.59     13129
           1       0.67      0.81      0.74     13417
           2       0.71      0.55      0.62     12899
           3       0.50      0.56      0.53     10856
           4       0.45      0.34      0.38     10690
           5       0.40      0.36      0.38      9669

   micro avg       0.56      0.56      0.56     70660
   macro avg       0.54      0.54      0.54     70660
weighted avg       0.56      0.56      0.55     70660
 samples avg       0.56      0.56      0.56     70660

0.5525567748281338
